In [1]:
import psycopg2
import random
import csv

In [2]:
def get_connection():
    return psycopg2.connect(
        host="localhost",
        database="5310 project",
        user="postgres",
        password="123"
    )

In [5]:
def generate_payment_orders(n=5000):
    conn = get_connection()
    cursor = conn.cursor()

    # Fetch payment_ids and order_ids
    cursor.execute("SELECT payment_id FROM payments")
    payment_ids = [row[0] for row in cursor.fetchall()]
    cursor.execute("SELECT order_id FROM customer_order")
    order_ids = [row[0] for row in cursor.fetchall()]

    if not payment_ids or not order_ids:
        print("Error: No payment_ids or order_ids found in the database.")
        cursor.close()
        conn.close()
        return

    print(f"Found {len(payment_ids)} payment_ids and {len(order_ids)} order_ids.")

    # Track used (payment_id, order_id) pairs
    used_pairs = set()
    
    # Generate unique random pairs of (payment_id, order_id)
    payment_orders = []
    while len(payment_orders) < n:
        payment_id = random.choice(payment_ids)
        order_id = random.choice(order_ids)
        pair = (payment_id, order_id)
        
        if pair not in used_pairs:
            payment_orders.append(pair)
            used_pairs.add(pair)
            
            # Insert into database
            try:
                cursor.execute(
                    "INSERT INTO payment_orders (payment_id, order_id) VALUES (%s, %s)",
                    (payment_id, order_id)
                )
                conn.commit()
            except psycopg2.IntegrityError as e:
                print(f"IntegrityError: {e}")
                conn.rollback()
            except Exception as e:
                print(f"Error: {e}")
                conn.rollback()

    cursor.close()
    conn.close()


    # Write the generated payment_order data to a CSV file
    with open('C:/Users/Yihua/Desktop/5310 project/payment_orders.csv', 'w', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(['payment_id', 'order_id'])
        csv_writer.writerows(payment_orders)

    print(f"Generated {len(payment_orders)} payment orders.")

if __name__ == "__main__":
    generate_payment_orders()

Found 2000 payment_ids and 5000 order_ids.
Generated 5000 payment orders.
